In [1]:
from sklearn.ensemble import *
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from pyHSICLasso import HSICLasso
import numpy as np
import lightgbm as lgb

In [2]:
labels = pd.read_csv('https://s3.amazonaws.com:443/drivendata/data/54/public/train_values.csv')
values = pd.read_csv('https://s3.amazonaws.com:443/drivendata/data/54/public/train_labels.csv"')
test = pd.read_csv('https://s3.amazonaws.com:443/drivendata/data/54/public/test_values.csv')

In [2]:
values['thal'] = pd.factorize(values['thal'])[0]
test['thal'] = pd.factorize(test['thal'])[0]
df = values.merge(labels, on='patient_id')

In [3]:
training_split = int(len(df) * 0.7)
y_train = df['heart_disease_present'].iloc[:training_split]
y_test = df['heart_disease_present'].iloc[training_split:]
x_train = df[[column for column in df.columns if column not in ['heart_disease_present', 'patient_id']]].iloc[:training_split]
x_test = df[[column for column in df.columns if column not in ['heart_disease_present', 'patient_id']]].iloc[training_split:]

In [5]:
lasso = HSICLasso()
lasso.input(x_train.values, y_train.values, featname=x_test.columns)
lasso.classification(10, M=1, B=0)
features = lasso.get_features()

Block HSIC Lasso B = 125.
M set to 1.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


In [33]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
lgb_total = lgb.Dataset(df[[column for column in df.columns if column not in ['heart_disease_present', 'patient_id']]], df['heart_disease_present'])



params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
#y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
test_pred = gbm.predict(test[[column for column in df.columns if column not in ['heart_disease_present', 'patient_id']]], num_iteration=gbm.best_iteration)

[1]	valid_0's l2: 0.251397	valid_0's l1: 0.494641
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.244355	valid_0's l1: 0.487951
[3]	valid_0's l2: 0.237778	valid_0's l1: 0.481136
[4]	valid_0's l2: 0.231991	valid_0's l1: 0.475045
[5]	valid_0's l2: 0.226678	valid_0's l1: 0.46895
[6]	valid_0's l2: 0.22059	valid_0's l1: 0.462193
[7]	valid_0's l2: 0.21503	valid_0's l1: 0.455439
[8]	valid_0's l2: 0.211407	valid_0's l1: 0.450314
[9]	valid_0's l2: 0.206861	valid_0's l1: 0.44459
[10]	valid_0's l2: 0.204095	valid_0's l1: 0.440359
[11]	valid_0's l2: 0.200562	valid_0's l1: 0.434842
[12]	valid_0's l2: 0.19762	valid_0's l1: 0.429648
[13]	valid_0's l2: 0.195201	valid_0's l1: 0.424757
[14]	valid_0's l2: 0.193243	valid_0's l1: 0.420149
[15]	valid_0's l2: 0.191554	valid_0's l1: 0.416131
[16]	valid_0's l2: 0.188872	valid_0's l1: 0.412963
[17]	valid_0's l2: 0.186421	valid_0's l1: 0.40996
[18]	valid_0's l2: 0.185753	valid_0's l1: 0.407357
[19]	valid_0's l2: 0.183825	valid_0'

In [40]:
test['heart_disease_present'] = test_pred
submission = test[['patient_id', 'heart_disease_present']]
submission.to_csv('submission1.csv', index=False)